# GENERAR GIF EVOLUCIÓN URBANÍSTICA

Instrucciones

In [2]:
!pip install transformers==4.27.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00


In [1]:
import numpy as np
from PIL import Image, ImageDraw
import imageio
import tensorflow as tf
import pandas as pd

import os
import cv2
import torch
import torchvision
from torch.optim import AdamW
from transformers import SegformerForSemanticSegmentation

from google.colab import files, drive

In [13]:
# Establezco una ruta absoluta a un directorio existente de mi Google Drive
ROOT_DIR = "/content/drive/MyDrive/TFM/dataset/"
AUX_DIR = "/content/drive/MyDrive/TFM/auxiliar"

IMAGES_DIR = os.path.join(ROOT_DIR, 'Images Google Earth')
IMAGES_DIR = os.path.join(ROOT_DIR,'Processed Images Mix')

# Conectamos con nuestro Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv(os.path.join(ROOT_DIR, 'class_dict.csv'))

classes = df['name']
palette = df[[' r', ' g', ' b']].values
id2label = classes.to_dict()
label2id = {v: k for k, v in id2label.items()}

In [4]:
# Carga del modelo preentrenado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
                                                         ignore_mismatched_sizes=True,
                                                         num_labels=len(id2label),
                                                         id2label=id2label,
                                                         label2id=label2id,
                                                         reshape_last_stage=True)

if torch.cuda.is_available():
    checkpoint = torch.load(os.path.join(AUX_DIR, 'best_model_checkpoint_III_5_2.pth'))
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    checkpoint = torch.load(os.path.join(AUX_DIR, 'best_model_checkpoint_III_5_2.pth'), map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])

optimizer = AdamW(model.parameters(), lr=0.00006)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at nvidia/mit-b5 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.classifier.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.classifier.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.b

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

> Borrar

El error indica que el objeto model no tiene un método llamado predict. Es probable que estés intentando usar un modelo de Hugging Face (específicamente SegformerForSemanticSegmentation). Hugging Face utiliza un enfoque diferente para predecir con modelos, y en lugar de usar un método predict, se utiliza el método forward (o simplemente llamando al modelo) después de asegurarse de que el modelo esté en modo de evaluación (model.eval()).

In [23]:
def predict_masks(images_list, model, device='cuda'):
    """
    Devuelve las máscaras predichas para una lista de imágenes.
    Las imágenes son divididas en porciones de 1000x1000 para la predicción.
    """
    masks = []
    #model = model.to(device)
    #model.eval()

    for img in images_list:
        subimages, original_size = split_into_subimages(img)
        submasks_list = []
        for subimage in subimages:
            pixel_values = torchvision.transforms.functional.to_tensor(subimage).unsqueeze(0).to(device)

            with torch.no_grad():
                outputs = model(pixel_values=pixel_values)
                #logits = outputs.logits.cpu()
                #upsampled_logits = torch.nn.functional.interpolate(outputs.logits, size=subimage.shape[-2:], mode="bilinear", align_corners=False)
                h, w = subimage.size
                upsampled_logits = torch.nn.functional.interpolate(outputs.logits, size=(h, w), mode="bilinear", align_corners=False)
                predicted = upsampled_logits.argmax(dim=1)

                pred_labels = predicted.detach().cpu().numpy()#.flatten()
                submasks_list.append(pred_labels)

            #upsampled_logits = torch.nn.functional.interpolate(logits, size=subimage.size[::-1], mode='bilinear', align_corners=False)
            #submask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()
            #submasks_list.append(submask)

        mask = combine_submasks(submasks_list, original_size)
        masks.append(mask)

    return masks

In [6]:
def detect_new_objects(prev_mask, current_mask, threshold=0.5):
    """Detecta los nuevos objetos en la máscara actual respecto a la máscara previa."""
    prev_bin = (prev_mask > threshold).astype(np.int8)
    curr_bin = (current_mask > threshold).astype(np.int8)

    new_objects = curr_bin - prev_bin

    return new_objects

In [7]:
def split_into_subimages(image, subimage_size=(1000, 1000)):
    """
    Divide una imagen en subimágenes de tamaño subimage_size.
    Si una imagen no es divisible exactamente por subimage_size, se aplica padding.
    """
    width, height = image.size
    new_width, new_height = subimage_size
    pad_width = (new_width - (width % new_width)) % new_width
    pad_height = (new_height - (height % new_height)) % new_height

    # Aplicar padding
    padded_image = Image.new(image.mode, (width + pad_width, height + pad_height), color=0)
    padded_image.paste(image, (0, 0))

    subimages = []
    for i in range(0, padded_image.width, new_width):
        for j in range(0, padded_image.height, new_height):
            subimage = padded_image.crop((i, j, i + new_width, j + new_height))
            subimages.append(subimage)

    return subimages, (width, height)

In [20]:
def combine_submasks(submasks, original_size, subimage_size=(1000, 1000)):
    width, height = original_size
    combined_mask = Image.new("L", (width, height))  # Crear una imagen en blanco y negro
    idx = 0
    new_width, new_height = subimage_size
    for i in range(0, width, new_width):
        for j in range(0, height, new_height):
            # Convertir el numpy array a imagen PIL
            submask_pil = Image.fromarray((submasks[idx].squeeze()*255).astype(np.uint8))
            combined_mask.paste(submask_pil, (i, j))
            idx += 1

    return combined_mask

In [31]:
def generate_gif(images_list, masks_list):
    """Genera un GIF con las máscaras y resalta nuevos objetos."""
    frames = []
    prev_mask = None
    for img, mask in zip(images_list, masks_list):
        mask_image = Image.new('RGB', img.size)
        draw = ImageDraw.Draw(mask_image)

        # Convertir la imagen 'mask' a un array numpy
        mask_array = np.asarray(mask)

        # Rellenar con máscaras existentes
        for x in range(img.width):
            for y in range(img.height):
                if mask_array[y, x] > 0.5:
                    draw.point((x, y), fill=(0, 255, 0))

        if prev_mask is not None:
            # Convertir la imagen 'prev_mask' a un array numpy
            prev_mask_array = np.asarray(prev_mask)

            new_objects = detect_new_objects(prev_mask_array, mask_array)

            for x in range(img.width):
                for y in range(img.height):
                    if new_objects[y, x] == 1:
                        draw.point((x, y), fill=(0, 0, 255))  # Azul para nuevos edificios
                    elif new_objects[y, x] == -1:
                        draw.point((x, y), fill=(255, 0, 0))  # Rojo para edificios que desaparecen

        prev_mask = mask_array
        frames.append(mask_image)

    imageio.mimsave('resultado.gif', frames, duration=1000, loop=0)

In [32]:
# Cargar las imágenes
image_paths = [os.path.join(IMAGES_DIR, "5845.tif"),
               os.path.join(IMAGES_DIR, "5847.tif"),
               os.path.join(IMAGES_DIR, "5849.tif"),
               os.path.join(IMAGES_DIR, "5852.tif")]
images_list = [Image.open(path) for path in image_paths]

# Predicción de máscaras
masks_list = predict_masks(images_list, model)

# Imprimir el número de edificios en cada imagen
for i, mask in enumerate(masks_list):
    mask_array = np.asarray(mask)
    number_of_objects = np.sum(mask_array > 0.5)
    print(f"Imagen {i+1}: {number_of_objects} edificios detectados.")

# Crear el GIF
generate_gif(images_list, masks_list)

Imagen 1: 41358 edificios detectados.
Imagen 2: 54906 edificios detectados.
Imagen 3: 54294 edificios detectados.
Imagen 4: 45711 edificios detectados.
